In [3]:
from tqdm import tqdm
from time import sleep

In [5]:
with tqdm(total=100) as pbar:
    for i in range(10):
        sleep(0.1)
        pbar.update(1)

 10%|█         | 10/100 [00:01<00:09,  9.16it/s]


In [1]:
from itertools import cycle


In [5]:
from torch.utils.data import Dataset, DataLoader

In [16]:
a = [1,2,3,4,5]
a = iter(a)

In [19]:

for i in range(10):
    try:
        print(next(a))
    except StopIteration as e:
        print(e)
        print('what is ithis')
        break

NameError: name 'prev' is not defined

In [30]:
a= """1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1."""

b= """0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1."""

In [32]:
a = [int(i) for i in a.replace('.', '').split(' ')]
b = [int(i) for i in b.replace('.', '').split(' ')]

In [33]:
from sklearn.metrics import matthews_corrcoef

In [35]:
matthews_corrcoef(b,a)

0.0

In [8]:
import pandas as pd

In [13]:

data = {
  "calories": [1,2,3,4,5,6,7,8,9],
  "duration": [50, 40, 45,4,5,6,7,8,9]
}

#load data into a DataFrame object:
df = pd.DataFrame(data)
bs = 4
for i in range(10):
    lnum = (i*bs)%len(df)
    unum = ((i+1)*bs)%len(df)

    if lnum < unum:
        print(df[lnum:unum])
    else:
        print(pd.concat([df[lnum:], df[:unum]]))
# a[9:]+a[:2]

   calories  duration
0         1        50
1         2        40
2         3        45
3         4         4
   calories  duration
4         5         5
5         6         6
6         7         7
7         8         8
   calories  duration
8         9         9
0         1        50
1         2        40
2         3        45
   calories  duration
3         4         4
4         5         5
5         6         6
6         7         7
   calories  duration
7         8         8
8         9         9
0         1        50
1         2        40
   calories  duration
2         3        45
3         4         4
4         5         5
5         6         6
   calories  duration
6         7         7
7         8         8
8         9         9
0         1        50
   calories  duration
1         2        40
2         3        45
3         4         4
4         5         5
   calories  duration
5         6         6
6         7         7
7         8         8
8         9         9
   calorie

In [7]:
max(1,2,3,4)

4